### 🛠️ Initialize Notebook Variables

**Only modify entries under _USER CONFIGURATION_.**

In [ ]:
import utils
from apimtypes import *

# ------------------------------
#    USER CONFIGURATION
# ------------------------------

rg_location = 'eastus2'
index       = 1
apim_sku    = APIM_SKU.BASICV2              # Options: 'BASICV2', 'STANDARDV2', 'PREMIUMV2'
deployment  = INFRASTRUCTURE.SIMPLE_APIM    # Options: 'AFD_APIM_PE', 'APIM_ACA', 'SIMPLE_APIM'
api_prefix  = ''            # Not defining a prefix for general as these APIs will live off the root
tags        = ['general']



# ------------------------------
#    SYSTEM CONFIGURATION
# ------------------------------

sample_folder    = 'general'
rg_name          = utils.get_infra_rg_name(deployment, index)
supported_infras = [INFRASTRUCTURE.AFD_APIM_PE, INFRASTRUCTURE.APIM_ACA, INFRASTRUCTURE.SIMPLE_APIM]
nb_helper        = utils.NotebookHelper(sample_folder, rg_name, rg_location, deployment, supported_infras, index = index, apim_sku = apim_sku)

# Define the APIs and their operations and policies

# API 1: Request Headers
rh_name                 = rh_path = f'{api_prefix}request-headers'
pol_request_headers_get = utils.read_policy_xml(REQUEST_HEADERS_XML_POLICY_PATH)
request_headers_get     = GET_APIOperation('Gets the request headers for the current request and returns them. Great for troubleshooting.', pol_request_headers_get)
request_headers         = API(rh_name, 'Request Headers', rh_path, 'API for request headers', operations = [request_headers_get], tags = tags)

# API 2: API ID: Extract and trace API Identifier
api_id_name             = api_id_path = f'{api_prefix}api-id'
pol_api_id_get          = utils.read_policy_xml(API_ID_XML_POLICY_PATH)
api_id_get              = GET_APIOperation('Gets the API identifier for the current request and traces it', pol_api_id_get)
api_id                  = API(api_id_name, 'API Identifier (api-42)', api_id_path, 'API for extracting and tracing API identifier', operations = [api_id_get], tags = tags)

# API 3: Correlation ID
corr_id_name            = corr_id_path = f'{api_prefix}correlation-id'
pol_corr_id_get         = utils.read_policy_xml(CORRELATION_ID_XML_POLICY_PATH)
corr_id_get             = GET_APIOperation('Ensures a correlation ID exists and returns it', pol_corr_id_get)
corr_id_api             = API(corr_id_name, 'Correlation ID', corr_id_path, 'Ensures a correlation ID header exists and is propagated', operations = [corr_id_get], tags = tags)

# Policy Fragments to deploy
pf_correlation = PolicyFragment('Correlation-Id', utils.read_policy_xml(utils.determine_shared_policy_path('pf-correlation-id.xml')), 'Ensures x-correlation-id header exists and is propagated.')
policy_fragments: list[PolicyFragment] = [pf_correlation]

# APIs Array
apis: List[API] = [request_headers, api_id, corr_id_api]

utils.print_ok('Notebook initialized')

### 🚀 Deploy Infrastructure and APIs

Creates the bicep deployment into the previously-specified resource group. A bicep parameters, `params.json`, file will be created prior to execution.

In [ ]:
# Build the bicep parameters
bicep_parameters = {
    'apis': {'value': [api.to_dict() for api in apis]},
    'policyFragments': {'value': [pf.to_dict() for pf in policy_fragments]}
}

# Deploy the sample
output = nb_helper.deploy_sample(bicep_parameters)

if output.success:
    # Extract deployment outputs for testing
    apim_gateway_url = output.get('apimResourceGatewayURL', 'APIM Gateway URL')
    apim_apis        = output.getJson('apiOutputs', 'APIs')

    utils.print_ok('Deployment completed successfully')
else:
    utils.print_error("Deployment failed!")
    raise SystemExit(1)

### ✅ Verify Deployment

Assert that the deployment was successful by making simple calls to APIM. 

❗️ If the infrastructure shields APIM and requires a different ingress (e.g. Azure Front Door), the request to the APIM gateway URl will fail by design. Obtain the Front Door endpoint hostname and try that instead.

In [ ]:
from apimrequests import ApimRequests
from apimtesting import ApimTesting

# Initialize testing framework
tests = ApimTesting("General Sample Tests", sample_folder, nb_helper.deployment, apim_apis)

# Preflight: Check if the infrastructure architecture deployment uses Azure Front Door. If so, assume that APIM is not directly accessible and use the Front Door URL instead.
endpoint_url = utils.test_url_preflight_check(deployment, rg_name, apim_gateway_url)

# 1) Request Headers
reqs = ApimRequests(endpoint_url, tests.get_key(rh_name))
output = reqs.singleGet('/request-headers', msg = 'Calling the Request Headers API. Expect 200.')
tests.verify('Host:' in output, True)

# 2) API ID
reqs = ApimRequests(endpoint_url, tests.get_key(api_id_name))
output = reqs.singleGet('/api-id', msg = 'Calling the API ID API. Expect 200.')
tests.verify(output, 'Extracted API ID: api-42')

# 3) Correlation ID
reqs = ApimRequests(endpoint_url, tests.get_key(corr_id_name))
# Provide an explicit correlation id to validate echo; then call without to ensure generation
explicit_cid = '11111111-2222-3333-4444-555555555555'
reqs.headers['x-correlation-id'] = explicit_cid
output = reqs.singleGet('/correlation-id', msg = 'Calling the Correlation ID API with explicit x-correlation-id. Expect 200.')
tests.verify(output.startswith('Correlation ID: '), True)
tests.verify(explicit_cid in output, True)

# Call again without header; should still return a GUID-like value
reqs.headers.pop('x-correlation-id', None)
output2 = reqs.singleGet('/correlation-id', msg = 'Calling the Correlation ID API without header. Expect APIM to generate one.')
tests.verify(output2.startswith('Correlation ID: '), True)

tests.print_summary()

utils.print_ok('All done!')